In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install jpype1
# pip install tabula-py

In [ ]:
pip install tabula-py==2.2.0

In [2]:
from tabula import read_pdf
import json
import re
import math
#import flash

In [3]:
# df = read_pdf("Fyrliste_HeleLandet.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_HeleLandet_20210425.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])

#df = read_pdf("Fyrliste_HeleLandet_29.11.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
df = read_pdf("Fyrliste_HeleLandet.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_Vest_20210428.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_MidtNorge_20210430.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_Nordland_20210501.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_TromsFinnmark_20210501.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df = read_pdf("Fyrliste_Svalbard_20210501.pdf", pages="all", multiple_tables=True, area=["%0,0,98,100"])
# df[0]["Område"]

/opt/homebrew/Caskroom/mambaforge/base/envs/fyrlysar/lib/python3.11/site-packages/tabula/io.py:688: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
/opt/homebrew/Caskroom/mambaforge/base/envs/fyrlysar/lib/python3.11/site-packages/tabula/io.py:688: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
/opt/homebrew/Caskroom/mambaforge/base/envs/fyrlysar/lib/python3.11/site-packages/tabula/io.py:688: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [4]:
def can_convert_to_float(string):
    try:
        result = float(string)
        return True
    except ValueError:
        return False
    
def extract_fyr_rows(rows):
    import re
    fyr_at_rows = []

    for i, row in enumerate(list(rows["Fyrnr."])):
        if not type(row) == str:
            continue
        #pattern = "\d{6}"
        #matches = re.findall(pattern, str(row))
        #if len(matches) > 0:
        if len(str(row)) >= 4 and str(row)[0:4].isdigit():
            fyr_at_rows.append(i)
    return fyr_at_rows

def parse_decimal(number):
    return float(number.replace(",", "."))


def extract_position(rows, start_row):
    latitude_string = rows["Posisjon"][start_row]
    latitude_degrees = float(latitude_string.split()[0])
    latitude_minutes = float(latitude_string.split()[1])
    # latitude = latitude_degrees + latitude_minutes / 60.0
    
    longitude_string = rows["Posisjon"][start_row+1]
    longitude_degrees = float(longitude_string.split()[0])
    longitude_minutes = float(longitude_string.split()[1])
    # longitude = longitude_degrees + longitude_minutes / 60.0
    return {
        "longitudeDeg": longitude_degrees,
        "longitudeMin": longitude_minutes,
        "latitudeDeg": latitude_degrees,
        "latitudeMin": latitude_minutes,
    }

def extract_name(rows, start_row):
    name = rows["Område"][start_row+1]
    if isinstance(name, str):
        return name
    return None

def extract_pattern(rows, start_row):
    pattern = rows["Karakter"][start_row]
    if isinstance(pattern, str):
        return pattern
    return None

def extract_height(rows, start_row):
    height = rows["Lys"][start_row]
    if isinstance(height, str):
        return parse_decimal(height)
    return None

def extract_sectors(rows, start_row, stop_row, has_bad_formatting):
    column_key = "Sektor - farge - retning - beskrivelse"
    if has_bad_formatting:
        column_key = "Unnamed: 0"
    r = list(rows[column_key][start_row:stop_row])
    sectors = []
    current_sector = None
    
    for row in r:
        if type(row) is not str:
            continue
        try:
            splitted = row.split()

            bad_row = splitted[0] in ["R", "G", "W"]

            if ( (splitted[0].isdigit() and len(splitted[0]) < 3 and splitted[1] in ["R", "G", "W"]) or splitted[0] in ["R", "G", "W"] ) and len(splitted) > 3:
                new_sector = True
                if current_sector is not None:
                    sectors.append(current_sector)
                current_sector = {}
            else:
                new_sector = False

            if new_sector:
                # print(row)
                index_offset = 0
                if bad_row:
                    index_offset = -1
                color = splitted[1 + index_offset]
                start = parse_decimal(splitted[2 + index_offset])
                stop  = parse_decimal(splitted[4 + index_offset])
                description = " ".join(splitted[ (5+index_offset) :])

                color_map = {
                    "R": "red",
                    "G": "green",
                    "W": "white",
                }

                current_sector["color"] = color_map[color]
                current_sector["start"] = start
                current_sector["stop"] = stop
                current_sector["description"] = description
            else:
                if current_sector:
                    description = row
                    current_sector["description"] += " "+description
        except:
            print("Error with sector")
            pass
    if current_sector is not None:
        sectors.append(current_sector)
    # print(f"Found {len(sectors)} sectors")
    return sectors

def extract_character(pattern):
    # Extract the light character
    # This method is intended to get input from extract_pattern above
    possible_classes = 'Iso|Fast|Oc|None|Fl|Q|VQ|FFl|LFl'

    if pattern is None:
        # TODO: Figure out how to display lighthouses without known pattern.
        return {
            "light_class": "None",
            "numflash": 0, 
            "extra_class": None, 
            "period": 0
        }
    
    light_classes = re.findall(possible_classes, pattern)
    
    if len(light_classes) > 2:
        # Don't think any lighthouse has more than two lightclasses
        print(f"{len(light_class)} light classes found", light_class)     
    
    light_class = light_classes[0]
    # TODO: Figure out what to do about the South Cardinal
    if len(light_classes) == 2:
        # some have more than one, like the South Cardinal: (Q or VQ) + LFl
        extra_class = light_classes[1]
    else:
        extra_class = None
        
#     numflash = 17
    # Extract number of flashes
    flash_string = re.findall('[(][\d+][)]', pattern)
#     print(f"{pattern=}, {light_class=}, {num_flash=}, {len(num_flash)=}")
    if len(flash_string) == 0:
#         print("len(num_flash) is zero.")
        numflash = 1
#         if light_class in ("Oc", "Fl", "FFl", "LFl" ):
#         # No number of flashes indicate 1 for Oc, Fl, FFl and LFl
#             numflash = 1
#         elif light_class in ("Q", "VQ", "UQ"):
#         # No number of flashes indicate continuos for Q, VQ (and UQ)
#             numflash = 0
#         elif light_class in ("Iso", "Fast"):
#         # No number of flashes are indicated for Iso, Fast
#             numflash = 0
#         else:
#             print(f"Error: Light class {light_class} not recognised.")
    elif len(flash_string) == 1:
        numflash = int(flash_string[0][1:-1])
    else:
        # len(flash_string) > 1
        print(f"{len(flash_string)} is  a strange number of flashes for lighthouse. {flash_string}")

    # Extract the period
    period = re.findall(' [\d]{0,2}s', pattern)
#     print(pattern, periode)
    if len(period) == 0:
        # Period not explicitly stated.
        if light_class == "Q":
            period = 1.0
        elif light_class == "VQ":
            period = 0.5
        elif light_class == "UQ":
            # Haven't seen ultra quick in the Norwegian Fyrliste yet.
            period = 0.25
        elif light_class == "Fast":
            period = 1.0
        else:
            print(f"Error: Light class {light_class} from pattern {pattern} should have an explicit period.")
    else:
#         print(periode[0][:-1])
        period = float(period[0][:-1])
    # TODO: Figure out what to do about "Fast lys yellow"
    # The pattern "Fast lys yellow" indicates that there is one single sector
    # and that this sector is yellow and 360 deg wide. Fyrliste seems to not
    # specify sectors for "Fast lys yellow". I guess:
    # if pattern = "Fast lys yellow":
    #     sectors = [{"color": "yellow", "start": 0.0, "stop": 360.0, "description": ""}]
    return {
        "light_class": light_class, 
        "numflash": numflash, 
        "extra_class": extra_class, 
        "period": period
    }

In [5]:
def extract_range(rows, start_row, has_bad_formatting):
    column_name = 'Lysvidde'
    if has_bad_formatting:
        column_name = 'Lysvidde Sektor - farge - retning - beskrivelse'
    red_range = green_range = white_range = 0
    if start_row < len(rows[column_name]):
        red_range = rows[column_name][start_row + 0]
    if start_row + 1 < len(rows[column_name]):
        green_range = rows[column_name][start_row + 1]
    if start_row + 2 < len(rows[column_name]):
        white_range = rows[column_name][start_row + 2]
    
    if type(red_range) == str:
        red_range = red_range.split(' ')[0] # Sometimes with bad formatting, two columns are wrongly merged so a string like "4,3 1" may occur
        red_range = red_range.replace(',', '.')
        if can_convert_to_float(red_range):
            red_range = float(red_range)
        else:
            red_range = 0
    if type(green_range) == str:
        green_range = green_range.split(' ')[0]
        green_range = green_range.replace(',', '.')
        if can_convert_to_float(green_range):
            green_range = float(green_range)
        else:
            green_range = 0
    if type(white_range) == str:
        white_range = white_range.split(' ')[0]
        white_range = white_range.replace(',', '.')
        if can_convert_to_float(white_range):
            white_range = float(white_range)
        else:
            white_range = 0
    if math.isnan(red_range):
        red_range = 0
    if math.isnan(green_range):
        green_range = 0
    if math.isnan(white_range):
        white_range = 0
    return red_range, green_range, white_range

lighthouses = []
all_stuff = []
for page in range(len(df)):
    rows = df[page]
    if "Fyrnr." not in rows.columns:
        print(f"Skipping page {page+1}")
        print(" Columns: ", rows.columns)
        continue
#     if page > 3:
#         break
    #print(f"Got columns {rows.columns}")
    has_bad_formatting = "Unnamed: 0" in rows.columns
    
    # all_stuff.extend(get_first_thingy(rows))
    
    fyr_indices = extract_fyr_rows(rows)
#     print("Working on page ", page+1)
#     print(fyr_indices)
    num_lighthouses_on_page = 0
    for i, start_row in enumerate(fyr_indices):
#         print(f"Working on start_row {start_row}")
        last_on_page = False
        if i == len(fyr_indices) - 1:
            last_on_page = True
            stop_row = len(rows)-1
        else:
            stop_row = fyr_indices[i+1]

        name = extract_name(rows, start_row)
        position = extract_position(rows, start_row)
        pattern = extract_pattern(rows, start_row)
        character = extract_character(pattern)
        height = extract_height(rows, start_row)
        sectors = extract_sectors(rows, start_row, stop_row, has_bad_formatting)
        red_range, green_range, white_range = extract_range(rows, start_row, has_bad_formatting)
        max_range = max(max(red_range, green_range), white_range) * 1852 # nautic mile to meter
#         print(f"Found {name} with max range {max_range}")
        lighthouse = {
            "latitude": position["latitudeDeg"] + position["latitudeMin"]/60,
            "longitude": position["longitudeDeg"] + position["longitudeMin"]/60,
            "pattern": pattern,
            "character": character,
            "height": height,
            "sectors": sectors,
            "name": name,
            "maxRange": max_range
        }
            
        lighthouses.append(lighthouse)
        num_lighthouses_on_page += 1
#         if last_on_page:
#              print(f"Last on page: {name}")

#     print(f"  Found {num_lighthouses_on_page} lighthouses on page {page+1}")
print(f"Found {len(lighthouses)} lighthouses")


Skipping page 1
 Columns:  Index(['-0-'], dtype='object')
Skipping page 2
 Columns:  Index(['Unnamed: 0'], dtype='object')
Skipping page 3
 Columns:  Index(['Unnamed: 0', 'GLOSSARY'], dtype='object')
Skipping page 4
 Columns:  Index(['Unnamed: 0', 'Unnamed: 1'], dtype='object')
Skipping page 5
 Columns:  Index(['Unnamed: 0'], dtype='object')
Skipping page 6
 Columns:  Index(['Unnamed: 0', 'KYSTVERKET'], dtype='object')
Skipping page 7
 Columns:  Index(['Unnamed: 0', 'Unnamed: 1'], dtype='object')
Skipping page 8
 Columns:  Index(['Unnamed: 0'], dtype='object')
Skipping page 9
 Columns:  Index(['Unnamed: 0'], dtype='object')
Skipping page 10
 Columns:  Index(['Merk: Den sjøfarende må imidlertid alltid', 'Unnamed: 0',
       'avgrenser. Vanligvis skal et sektorlys også'],
      dtype='object')
Skipping page 11
 Columns:  Index(['LYSKARAKTERER\rLIGHT CHARACTERS',
       'Internasjonale\rforkortelser\rInternational\rabbreviations',
       'De viktigste lyskarakterer skjematisk framstilt\rT

In [7]:
with open("../lighthouses.qml", "w") as f:
    json_lines = json.dumps(lighthouses, indent=2).split('\n')
    f.write("""import QtQuick

Rectangle {
    property var lighthouses
    property string jsonString: '""")
    for line in json_lines:
        line = line.replace("'", "")
        line = line.encode('utf-8').decode('unicode-escape')
        f.write(line+'\ \n')
    f.write("'\n")
    f.write("""    Component.onCompleted:  {
        lighthouses = JSON.parse(jsonString)
    }
}""")

# print("Dumped json.")

In [ ]:
for light in lighthouses:
    print(f"{light}")

In [ ]:
patterns = {}
# Count the occurence of various light characters
for light in lighthouses:
#     print(light['pattern'])
    pat = light['pattern']
#     print(pat, type(pat))
    if pat is not None:
        pat = re.sub('\s[WRG]+\s', ' ', pat)
    else:
        pat = "None"
    if pat in patterns.keys():
        patterns[pat] += 1
    else:
        patterns[pat] = 1
# print(patterns)

# Sort light characters by decending popularity and print.
patterns_sorted_popular = {}
sorted_keys_by_value = sorted(patterns, key=patterns.get)[::-1]
# print(sorted_keys_by_value)
for w in sorted_keys_by_value:
    patterns_sorted_popular[w] = patterns[w]
for k,v in patterns_sorted_popular.items():
    print(f"{str(k):<16}{v:>4}")
print("")
patterns_sorted = sorted(patterns.items())
# Print patterns alphabetically
for k, v in sorted(patterns.items()):
    print(f"{k:<16}{v:>4}")

In [ ]:

patterns_collected = {}
for pattern in patterns_sorted_popular.keys():
    if pattern is None:
        pattern = "None"
    if pattern in patterns_collected.keys():
        patterns_collected[pat] += patterns_sorted_popular[pattern]
    else:
        patterns_collected[pat] = patterns_sorted_popular[pattern]
for k,v in patterns_collected.items():
    print(f"{str(k):<9}{v:>3}")

    

In [ ]:
for pat in patterns_sorted_popular.keys():
#     print(pat)
#     if pat is not None:
    print(f"{str(pat):<18}", extract_character(pat))

In [ ]:
for light in lighthouses:
    print(f"{light}")

In [ ]:
import flash
import time
import math

check_house = "Bergholmen, NØ"

for light in lighthouses:
    if light["name"] == check_house:
        flash_on_off = flash.mk_flash(light["character"]["light_class"], 
                                      light["character"]["numflash"], 
                                      light["character"]["period"],
                                      extra_light_class=light["character"]["extra_class"]
                                     )
        pattern = light["pattern"]
        period = light["character"]["period"]
        print(flash_on_off)
        print(len(flash_on_off))
        koeff = len(flash_on_off) / period
#         now = int(math.floor(time.time() % period) * koeff)
#         print(now)
        print(f"Lighthouse {check_house} with pattern {pattern} goes:")
        for n in range(200):
            time.sleep(0.1)
            now = int(math.floor((time.time() % period) * koeff))
#             print(f"\r{now})
            if flash_on_off[now]:
                print(f"\r{n:4} {now:2} Bright light, bright light!", end="")
#                 print(f"{n:4} {now:2} Bright light, bright light!")
            else:
                print(f"\r{n:4} {now:2} Dark...                    ", end="")
#                 print(f"{n:4} {now:2} Dark...                    ")
            